In [129]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import imutils.paths as path
from tqdm import tqdm

In [130]:
PATH = 'data_train'
imagePaths = sorted(list(path.list_images(PATH)))

In [131]:
data = []
for i in tqdm(imagePaths,desc="load"):
    imgg=cv.imread (i)
    img = cv.cvtColor(imgg, cv.COLOR_BGR2GRAY)
    a = cv.resize (img, (128, 128))
    data.append (a)

load: 100%|██████████| 205/205 [00:01<00:00, 182.69it/s]


In [132]:
 def derajat0 (img):
        # Menghitung nilai piksel maksimum dalam citra
        max = np.max(img)
        
        # Inisialisasi matriks nol dengan ukuran (max+1) x (max+1)
        imgTmp=np.zeros([max+1,max+1])
        
        # Iterasi melalui piksel-piksel citra
        for i in range (len(img)):
            for j in range (len(img[i])-1):
                
                 # Menghitung pasangan nilai intensitas piksel dan menyimpannya dalam imgTmp
                imgTmp[img[i,j],img[i,j+1]] +=1
                
        # Menghitung transpos matriks imgTmp        
        transpos= np.transpose(imgTmp)
        
        # Menjumlahkan matriks imgTmp dengan transpos-nya
        data= imgTmp+transpos
        
        # Menghitung total elemen dalam matriks data
        tmp=0
        for i in range (len(data)):
            for j in range (len(data)):
                tmp+=data[i,j]
        # Normalisasi matriks data dengan membagi setiap elemen oleh total elemen        
        for i in range (len (data)):
            for j in range(len (data)):
                data[i,j]/=tmp
                
        # Mengembalikan matriks hasil normalisasi
        return data

In [133]:
def derajat45 (img):
    max = np.max(img)
    imgTmp=np.zeros([max+1, max+1])
    for i in range (len (img)-1):
        for j in range (len (img[i])-1):
            imgTmp[img[i+1,j],img[i,j+1]] += 1
            
    transpos= np.transpose(imgTmp)
    data= imgTmp+transpos
    
    tmp=0 
    for i in range (len(data)):
        for j in range (len(data)):
            tmp+=data[i,j]
            
    for i in range (len (data)):
        for j in range (len (data)):
            data[i,j]/=tmp
    return data        

In [134]:
def derajat90 (img):
    max = np.max(img)
    imgTmp=np.zeros([max+1, max+1])
    for i in range (len (img)-1):
        for j in range (len (img[i])-1):
            imgTmp[img[i+1,j],img[i,j+1]] += 1
            
    transpos= np.transpose(imgTmp)
    data= imgTmp+transpos
    
    tmp=0 
    for i in range (len(data)):
        for j in range (len(data)):
            tmp+=data[i,j]
            
    for i in range (len (data)):
        for j in range (len (data)):
            data[i,j]/=tmp
    return data        

In [135]:
def derajat135 (img):
    max = np.max(img)
    imgTmp=np.zeros([max+1, max+1])
    for i in range (len (img)-1):
        for j in range (len (img[i])-1):
            imgTmp[img[i+1,j],img[i,j+1]] += 1
            
    transpos= np.transpose(imgTmp)
    data= imgTmp+transpos
    
    tmp=0
    for i in range (len(data)):
        for j in range (len(data)):
            tmp+=data[i,j]
            
    for i in range (len (data)):
        for j in range (len (data)):
            data[i,j]/=tmp
    return data        

In [136]:
hasil=[]
for i in tqdm (range(len (data)), desc='GLCM'):
    dat=[]
    dat.append(derajat0 (data[i]))
    dat.append(derajat45 (data[i]))
    dat.append(derajat90 (data[i]))
    dat.append(derajat135 (data[i]))
    hasil.append(dat)

GLCM: 100%|██████████| 205/205 [00:55<00:00,  3.69it/s]


In [72]:
def contras (data):
    contras=0
    for i in range (len (data)):
        for j in range (len (data)):
            contras+=data[i,j]*pow(i-j,2)
    return contras

In [73]:
def correrlation (data):
    
    i, j = np.indices(data.shape)
    mu_x = np.sum(i * data)
    mu_y = np.sum(j * data)
    sigma_x = np.sqrt(np.sum((i - mu_x)**2 * data))
    sigma_y = np.sqrt(np.sum((j - mu_y)**2 * data))
    
    corr=0
    for i in range(len(data)):
        for j in range(len(data)):
            corr += data[i,j] * (i - mu_x) * (j - mu_y)
            
    corr /= (sigma_x * sigma_y)
            
    return corr

In [74]:
def homogenitas (data):
    homogen=0
    for i in range (len (data)):
        for j in range (len (data)):
            homogen+=data[i,j]*(1+(pow(i-j,2)))
    return homogen

In [75]:
def energi (data):
    ener=0
    for i in range (len (data)):
        for j in range (len (data)):
            ener += data[i,j]**2
    return ener

In [76]:
data0energi=[]
data=[]
x=['0','45','90','135']
data45=[]
data90=[]
data135=[]
hasilnya=[]

for j in tqdm (range(len(hasil)),desc="Ekstraksi"):
    da=[]
    da.append(imagePaths[j])
    for i in hasil[j]:
        dx=energi (i)
        da.append(dx)
        
        dh=homogenitas (i)
        da.append(dh)
        
        den=correrlation (i)
        da.append(den)
        
        dco=contras (i)
        da.append(dco)
        
        hasilnya.append(da)

Ekstraksi: 100%|██████████| 205/205 [02:14<00:00,  1.52it/s]


In [32]:
namatabel = ['file', 'energy_0', 'homogenity_0', 'correlation_0', 'contrast_0',
             'energy_45', 'homogenity_45', 'correlation_45', 'contrast_45', 
             'energy_90', 'homogenity_90', 'correlation_90', 'contrast_90',
             'energy_135', 'homogenity_135', 'correlation_135', 'contrast_135']
df = pd.DataFrame(hasilnya, columns=namatabel)

In [33]:
df.head()
df.shape

(820, 17)

In [34]:
df

,file,energy_0,homogenity_0,correlation_0,contrast_0,energy_45,homogenity_45,correlation_45,contrast_45,energy_90,homogenity_90,correlation_90,contrast_90,energy_135,homogenity_135,correlation_135,contrast_135
0,data_train\Normal-16.png,0.004705,167.975763,0.980697,166.975763,0.004009,221.093558,0.974256,220.093558,0.004009,221.093558,0.974256,220.093558,0.004009,221.093558,0.974256,220.093558
1,data_train\Normal-16.png,0.004705,167.975763,0.980697,166.975763,0.004009,221.093558,0.974256,220.093558,0.004009,221.093558,0.974256,220.093558,0.004009,221.093558,0.974256,220.093558
2,data_train\Normal-16.png,0.004705,167.975763,0.980697,166.975763,0.004009,221.093558,0.974256,220.093558,0.004009,221.093558,0.974256,220.093558,0.004009,221.093558,0.974256,220.093558
3,data_train\Normal-16.png,0.004705,167.975763,0.980697,166.975763,0.004009,221.093558,0.974256,220.093558,0.004009,221.093558,0.974256,220.093558,0.004009,221.093558,0.974256,220.093558
4,data_train\Normal-181.png,0.000386,157.010212,0.980389,156.010212,0.000299,206.705313,0.973824,205.705313,0.000299,206.705313,0.973824,205.705313,0.000299,206.705313,0.973824,205.705313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815,data_train\Tuberculosis-98.png,0.001360,116.642040,0.932832,115.642040,0.001170,229.208630,0.865524,228.208630,0.001170,229.208630,0.865524,228.208630,0.001170,229.208630,0.865524,228.208630
816,data_train\Tuberculosis-99.png,0.001265,111.194390,0.926544,110.194390,0.001010,194.399095,0.869912,193.399095,0.001010,194.399095,0.869912,193.399095,0.001010,194.399095,0.869912,193.399095
817,data_train\Tuberculosis-99.png,0.001265,111.194390,0.926544,110.194390,0.001010,194.399095,0.869912,193.399095,0.001010,194.399095,0.869912,193.399095,0.001010,194.399095,0.869912,193.399095
818,data_train\Tuberculosis-99.png,0.001265,111.194390,0.926544,110.194390,0.001010,194.399095,0.869912,193.399095,0.001010,194.399095,0.869912,193.399095,0.001010,194.399095,0.869912,193.399095


In [35]:
df.to_csv(r'data_train.csv', index=False)

In [1]:
#KLASIFIKASI SVM

In [2]:
train_path = r'C:\Users\ASUS\datatbc\data_train.csv'
test_path = r'C:\Users\ASUS\datatbc\data_test.csv'

In [3]:
#membaca data training dan data testing

import pandas as pd

data_train = pd.read_csv(train_path, sep=';')
data_test = pd.read_csv(test_path, sep=';')

print(data_train.columns.tolist())

['diagnosis', 'energy_0', 'homogenity_0', 'correlation_0', 'contrast_0', 'energy_45', 'homogenity_45', 'correlation_45', 'contrast_45', 'energy_90', 'homogenity_90', 'correlation_90', 'contrast_90', 'energy_135', 'homogenity_135', 'correlation_135', 'contrast_135']


In [4]:
label_train = data_train['diagnosis'].to_numpy()

fitur_train = data_train[[ 'energy_0', 'homogenity_0', 'correlation_0', 'contrast_0',
             'energy_45', 'homogenity_45', 'correlation_45', 'contrast_45', 
             'energy_90', 'homogenity_90', 'correlation_90', 'contrast_90',
             'energy_135', 'homogenity_135', 'correlation_135', 'contrast_135']].to_numpy()

print(fitur_train)

[[2.1000000e-03 8.1251480e+01 9.8601000e-01 ... 1.1780966e+02
  9.7935000e-01 1.1680966e+02]
 [2.1000000e-03 8.1251480e+01 9.8601000e-01 ... 1.1780966e+02
  9.7935000e-01 1.1680966e+02]
 [2.1000000e-03 8.1251480e+01 9.8601000e-01 ... 1.1780966e+02
  9.7935000e-01 1.1680966e+02]
 ...
 [1.2600000e-03 1.1119439e+02 9.2654000e-01 ... 1.9439909e+02
  8.6991000e-01 1.9339909e+02]
 [1.2600000e-03 1.1119439e+02 9.2654000e-01 ... 1.9439909e+02
  8.6991000e-01 1.9339909e+02]
 [1.2600000e-03 1.1119439e+02 9.2654000e-01 ... 1.9439909e+02
  8.6991000e-01 1.9339909e+02]]


In [5]:
#membagi data training menjadi training set dan testing set (validation) untuk membuat model dan menguji akurasi model yang dibuat

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(fitur_train, label_train, random_state=0)

In [6]:
#Pendefinisian Kernel

from sklearn.svm import SVC

svc = SVC(kernel='rbf')
svc.fit(x_train, y_train)

SVC()

In [7]:
#menguji model clasifier yang dibuat ke testing set 
y_pred = svc.predict(x_test)

In [8]:

#menghitung akurasi testing set
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(y_test, y_pred) .ravel()
accuracy = (tn + tp) / (tn+fp+fn+tp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2 * (precision * recall) / (precision + recall)

print("Metrics on Test Data:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


Metrics on Test Data:
Accuracy: 0.7272727272727273
Precision: 0.7647058823529411
Recall: 0.6610169491525424
F1 Score: 0.7090909090909091


In [9]:
y_pred_train = svc.predict(x_train)


In [10]:
# Menghitung metrik evaluasi pada data latih
tn_train, fp_train, fn_train, tp_train = confusion_matrix(y_train, y_pred_train).ravel()
accuracy_train = (tn_train + tp_train) / (tn_train + fp_train + fn_train + tp_train)
precision_train = tp_train / (tp_train + fp_train)
recall_train = tp_train / (tp_train + fn_train)
f1_score_train = 2 * (precision_train * recall_train) / (precision_train + recall_train)

print("Metrics on Train Data:")
print("Accuracy:", accuracy_train)
print("Precision:", precision_train)
print("Recall:", recall_train)
print("F1 Score:", f1_score_train)

Metrics on Train Data:
Accuracy: 0.6590909090909091
Precision: 0.6835164835164835
Recall: 0.5901328273244781
F1 Score: 0.6334012219959266


In [11]:
#menerapkan model ke data testing untuk mendapatkan kelasnya
fitur_test = data_test[['energy_0', 'homogenity_0', 'correlation_0', 'contrast_0',
             'energy_45', 'homogenity_45', 'correlation_45', 'contrast_45', 
             'energy_90', 'homogenity_90', 'correlation_90', 'contrast_90',
             'energy_135', 'homogenity_135', 'correlation_135', 'contrast_135']]
y_pred_test = svc.predict(fitur_test)
print(y_pred_test)

['N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'T' 'T' 'T' 'T' 'N' 'N' 'N' 'N' 'T' 'T'
 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'T' 'T' 'T' 'T' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'T' 'T' 'T' 'T' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'T' 'T' 'T' 'T' 'N' 'N' 'N' 'N'
 'T' 'T' 'T' 'T' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'N' 'N' 'N' 'N' 'T' 'T' 'T' 'T'
 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T'
 'T' 'T' 'T' 'T' 'T' 'T' 'N' 'N' 'N' 'N' 'N' 'N' 'N

C:\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


In [18]:
import pickle

# Save the model to a file
with open('model_tbc.pkl', 'wb') as file:
    pickle.dump(svc, file)
